<a href="https://colab.research.google.com/github/dmtmcms01/13th_AI_ASSIGNMENT/blob/main/4%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torch.nn as nn
import numpy
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.init
import matplotlib.pyplot as plt

In [15]:
rate=0.001
epoch=20
batch_size = 64

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [21]:
Fashionmnist_train = datasets.FashionMNIST(
    root='FashionMNIST/',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)

Fashionmnist_test = datasets.FashionMNIST(
    root='FashionMNIST/',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

In [25]:
data = DataLoader(Fashionmnist_train, batch_size=batch_size, shuffle=True)
test_data = DataLoader(Fashionmnist_test, batch_size=batch_size)

In [18]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2), # B,32,14,14

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2), # B,64,7,7

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2), # B,128,3,3

            nn.Flatten(),

            nn.Linear(128 * 3* 3, 256, bias=True),
            nn.Linear(256, 10)
        )

    def forward(self, x):
      return self.model(x)

In [19]:
model = CNN().to(device)
cr = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=rate)

In [27]:
for epoch in range(epoch):
  model.train()
  cor = 0
  tot = 0
  cost = 0
  for image, label in data:
    image= image.to(device)
    label= label.to(device)

    optimizer.zero_grad()

    pred = model(image)
    loss = cr(pred, label)
    loss.backward()
    optimizer.step()
    cost += loss / len(data)

    cor += (torch.argmax(pred, dim=1)==label).float().sum()
    tot += label.size(0)
    acc = 100 * cor / tot
  print(f"[Epoch {epoch+1}] Cost: {cost:.2f}, train_acc: {acc:.2f}")



[Epoch 1] Cost: 0.05, train_acc: 98.30
[Epoch 2] Cost: 0.04, train_acc: 98.46
[Epoch 3] Cost: 0.04, train_acc: 98.45
[Epoch 4] Cost: 0.04, train_acc: 98.59
[Epoch 5] Cost: 0.04, train_acc: 98.58
[Epoch 6] Cost: 0.03, train_acc: 98.80
[Epoch 7] Cost: 0.03, train_acc: 98.81
[Epoch 8] Cost: 0.04, train_acc: 98.73
[Epoch 9] Cost: 0.03, train_acc: 98.91
[Epoch 10] Cost: 0.04, train_acc: 98.65
[Epoch 11] Cost: 0.03, train_acc: 98.93
[Epoch 12] Cost: 0.03, train_acc: 98.89
[Epoch 13] Cost: 0.03, train_acc: 98.92
[Epoch 14] Cost: 0.03, train_acc: 99.01
[Epoch 15] Cost: 0.03, train_acc: 98.99
[Epoch 16] Cost: 0.03, train_acc: 99.01
[Epoch 17] Cost: 0.03, train_acc: 99.03
[Epoch 18] Cost: 0.03, train_acc: 99.05
[Epoch 19] Cost: 0.03, train_acc: 99.10


In [28]:
model.eval()

with torch.no_grad():
  cor = 0
  tot = 0
  for image, label in test_data:
    image = image.to(device)
    label = label.to(device)

    pred = model(image)
    loss = cr(pred, label)

    cor += (torch.argmax(pred, dim=1)==label).float().sum()
    tot += label.size(0)
    acc = 100 * cor / tot
  print(f"Accuracy: {acc:.2f}%")



Accuracy: 91.16%
